In [ ]:
# Load a spacy model and chekc if it has ner
import spacy
nlp=spacy.load('en_core_web_sm')
from spacy.tokens import Span
import difflib

In [ ]:
# Getting Locations
import numpy as np
import pandas as pd
data = pd.read_csv('task_1_information_extraction_valid.tsv', sep='\t')
print(len(data))
data = data[data['NOTES'].notna()]
data = data.reset_index(drop=True)
data.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nltk
import spacy
  
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
! pip install locationtagger
import locationtagger

In [ ]:
import numpy as np

# Get Location from text, a text can also contain multiple locations
def getlocation():
    countries = []
    regions = []
    cities = []
    for i in range(len(data)):
        line = data['NOTES'][i]

        if (i % 1000) == 0 and (i != 0):
            print("Finished Sentense :", i)

        place_entity = locationtagger.find_locations(text = line)

        # if param == 'countries':
        countries.append(place_entity.countries)
        
        # if param == 'regions':
        regions.append(place_entity.regions)
        
        # if param == 'cities':
        cities.append(place_entity.cities)

    return countries, regions, cities

countries, regions, cities = getlocation()

In [ ]:
# Saving Arrays to files
# with open("countries.txt", "w") as f:
#     for s in countries:
#         f.write(str(s) +"\n")

# with open("regions.txt", "w") as f:
#     for s in regions:
#         f.write(str(s) +"\n")

# with open("cities.txt", "w") as f:
#     for s in cities:
#         f.write(str(s) +"\n")

In [ ]:
# Loading arrays from files
# countries = []
# with open("countries.txt", "r") as f:
#   for line in f:
#     countries.append(line.strip())

# regions = []
# with open("regions.txt", "r") as f:
#   for line in f:
#     regions.append(line.strip())

# cities = []
# with open("cities.txt", "r") as f:
#   for line in f:
#     cities.append(line.strip())

In [ ]:
print(countries)

In [ ]:
cities[1].strip()

In [ ]:
def getcounts(Locations):
    count = 0
    for i in range(len(Locations)):
        if not Locations[i]:
            count = count + 1

    print(len(Locations))
    print("Total Empty Locations: ", count)

getcounts(countries)
getcounts(regions)
getcounts(cities)

In [ ]:
# Using Spacy on Empty Locations
def spacy(Locations):
    for i in range(len(Locations)):
        if not Locations[i]:
            doc = nlp(data['NOTES'][i])
            loc = []
            for ent in doc.ents:  
                if ent.label_ in 'LOC':
                    Locations[i] = ent.text
                    # loc.append(ent.text)
                    continue
                elif ent.label_ in 'GPE':
                    # loc.append(ent.text)
                    Locations[i] = ent.text
                    continue
spacy(countries)
spacy(regions)
spacy(cities)

In [ ]:
Actual_Locations = data['LOCATION']

In [ ]:
# Accuracy
def getAccuracy(Locations):
    correct = 0
    for i in range(len(Locations)):

        if not Locations[i]:
            continue

        if difflib.SequenceMatcher(None,Locations[i],Actual_Locations[i]).ratio() >= 0.3:
            correct += 1

    # print(correct)
    print("Accuracy :", correct/len(Locations))

getAccuracy(countries)
# getAccuracy(regions)
# getAccuracy(cities)

In [ ]:
# Getting Fatalities
Numbers = []
import numpy as np
for i in range(len(data)):
    line = data['NOTES'][i]
    doc = nlp(line)

    if (i % 1000) == 0 and (i != 0):
        print("Finished Sentense :", i)

    num = []
    for ent in doc.ents:  
        if ent.label_ in ['CARDINAL']:
            num.append(ent.text)
        else:
            num.append(0)

    Numbers.append(num)

In [ ]:
len(Numbers)

In [ ]:
# Removing sentences which return 0 cardinals 
for i in range(len(Numbers)):
    if Numbers[i] != []:
        F1.append(Numbers[i][0])
    else:
        F1.append(0)

In [ ]:
# Removing dates, converting string to lower from sentences 
from re import search
F2 = F1.copy()
for i in range(len(F2)):
    if type(F2[i]) == str:
        F2[i] = F2[i].lower()
        if search('/', F2[i]) or search(':', F2[i]):
            F2[i] = 0
    else:
        F2[i] = F2[i]

In [ ]:
F2

In [ ]:
# Splitting bigger strings
F3 = F2.copy()
for i in range(len(F3)):
    if type(F3[i]) == str:
        x = F3[i].split()
        if len(x) > 1:
            for j in x:
                doc = nlp(j)
                for ent in doc.ents:  
                    if ent.label_ in ['CARDINAL']:
                        F3[i] = ent.text
                    else:
                        F3[i] = 0
    else:
        F3[i] = F2[i]

In [ ]:
F3

In [ ]:
# Convert strings to numbers
# !pip install word2number
# !pip install pattern
from word2number import w2n
from pattern.text.en import singularize
F4 = F3.copy()
for i in range(len(F4)):
    if type(F4[i]) == str:
        try:
            F4[i] = w2n.word_to_num(F4[i])
        except Exception:
            pass
    else:
        F4[i] = F3[i]

In [ ]:
len(F4)

In [ ]:
# Saving Predicted Fatalities
# with open("fatalities.txt", "w") as f:
#     for s in F4:
#         f.write(str(s) +"\n")

In [ ]:
# F4 = []
# with open("fatalities.txt", "r") as f:
#   for line in f:
#     F4.append(line.strip())

In [ ]:
Actual_Fatalities = data['FATALITIES']

In [ ]:
# Calculating Accuracy
correct = 0
result1 = []
result2 = []
for i in range(len(Actual_Fatalities)):
    try:
        if Actual_Fatalities[i] == int(F4[i]):
            correct += 1
            result1.append([i, Actual_Fatalities[i], F4[i]])
    except Exception:
            pass

print("Accuracy")
print(correct/len(Actual_Fatalities))